In [7]:
from pathlib import Path
from transformers import AutoTokenizer, AutoModel
import torch
from tqdm.auto import tqdm
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(device)

cuda


In [8]:
tok_mod_path = Path('/mnt/data/factcheck/tweets/similarity/models')

models_full_names = [
    'AIDA-UPM/mstsb-paraphrase-multilingual-mpnet-base-v2',
    'AIDA-UPM/MSTSb_paraphrase-multilingual-MiniLM-L12-v2',
    'AIDA-UPM/MSTSb_stsb-xlm-r-multilingual',
    'AIDA-UPM/MSTSb_paraphrase-xlm-r-multilingual-v1'
]

models_simple_names = [
    'para_mpnet',
    'para_minilm',
    'stsb_xlm',
    'para_xlm'
]

def get_tokenizer_path(name):
    return tok_mod_path / f'{name}_tokenizer'

def get_model_path(name):
    return tok_mod_path / f'{name}_model'

### Load Files

In [9]:
import json

def load_dict(flnm):
    with open(flnm, 'rt') as f:
        data = json.load(f)
    data = {int(k): v for k, v in data.items()}
    return data

all_to_text = load_dict(data_dir / 'all_text.json')
all_to_wrap = load_dict(data_dir / 'all_wrap.json')
all_to_vecno = load_dict(data_dir / 'all_vecno.json')

print(len(all_to_text), len(all_to_wrap), len(all_to_vecno))

203057 203057 203057


### Compute and Save Vectors

In [26]:
import numpy as np

def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

from torch.utils.data import DataLoader, TensorDataset

def get_lengths(claims, tokenizer_path):
    tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)
    
    lengths = np.zeros(len(claims))
    for i, c in enumerate(claims):
        encoded_claims = tokenizer(c, padding=False, truncation=False, return_tensors='pt')
        lengths[i] = encoded_claims['input_ids'].shape[1]
    return lengths

tokenizer_path = get_tokenizer_path(models_simple_names[0])
lengths = get_lengths(claims, tokenizer_path)

# claims = list(all_to_text.values())
# for i, name in enumerate(models_simple_names[:1]):
#     tokenizer_path = get_tokenizer_path(name)
#     tokenized = tokenize(claims, tokenizer_path)

In [38]:
print('total number of tweets:', lenghts.shape[0])
print('number of tweets exceeding max length:', (lengths > 126).sum())
print('number of tweets exceeding max length by more than 10:', (lengths > 136).sum())
toolong = lengths[lengths > 126]
for i in range(127, int(toolong.max().item()) + 1):
    print(i, (toolong == i).sum())

total number of tweets: 203057
number of tweets exceeding max length: 548
number of tweets exceeding max length by more than 10: 334
127 30
128 31
129 21
130 26
131 20
132 21
133 14
134 16
135 23
136 12
137 18
138 16
139 13
140 8
141 7
142 9
143 6
144 10
145 6
146 8
147 7
148 6
149 9
150 8
151 6
152 6
153 9
154 13
155 5
156 5
157 6
158 7
159 13
160 6
161 4
162 5
163 5
164 1
165 5
166 2
167 2
168 4
169 1
170 4
171 3
172 3
173 3
174 0
175 3
176 1
177 1
178 3
179 3
180 0
181 1
182 2
183 1
184 3
185 0
186 1
187 0
188 0
189 3
190 1
191 1
192 2
193 1
194 1
195 0
196 2
197 1
198 0
199 1
200 1
201 0
202 1
203 0
204 0
205 0
206 0
207 0
208 0
209 0
210 0
211 0
212 0
213 0
214 1
215 0
216 0
217 0
218 1
219 0
220 0
221 1
222 1
223 2
224 1
225 1
226 0
227 1
228 0
229 0
230 1
231 0
232 1
233 1
234 0
235 0
236 1
237 0
238 2
239 1
240 0
241 1
242 2
243 1
244 3
245 1
246 3
247 0
248 2
249 2
250 0
251 1
252 1
253 2
254 1
255 3
256 0
257 0
258 1
259 0
260 0
261 2
262 0
263 1
264 0
265 1
266 0
267 0
268 0